# IMPORTS

In [ ]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

## Setting hyperparameters and creating the file _config_

Name of the .json file where the network configuration will be stored. Don't change this value.

In [ ]:
filename = '/config_network.json'

## Setting the main parameters of the network

Print default configuration

In [ ]:
default_configuration()

We then set the main parameters and the architecture of the network that we want to change, compared to the default configuration above.

L_struct = [{'structure':[[5,5,5], [3,3,3], [3,3,3], [3,3,3]], 'features_augmentation':'x2', 'first_num_features':16}]

dict_params = {'trainingset': ['SEM_3c_512'],
               'trainingset_patchsize': 512,
               'learning_rate_decay_period':24000
              }

config_list = grid_config(L_struct, dict_params)

In [ ]:
config_list = {'gridblockperone':default_configuration()}

## Actually training the model

By default, the name is the current date. You can uncomment the second line dir_name if you want to give a specific name to the folder, but beware: it will erase the contents of the folder if it already exists !

The model data will be stored in a folder with this name, as well as the summaries that are going to be readable on tensorboard.

In [ ]:
#dir_name = time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S") 
#dir_name = 'cv_2c_nw_d5_2_b8_p10-10'
#dir_name += '_' + str(int(time.time()))[-4:]

for dir_name, config in config_list.iteritems():
    pass
path_model = os.path.join('../models/', dir_name)

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
path_model_init = None 


# Specify here the path to the initial model if needed.

if os.path.exists(path_model+filename): # if there is already a configfile
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_model+filename, 'w') as f:
        json.dump(config, f, indent=2)
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
        
        
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config_network)
path_trainingset = os.path.join('../data/', config_network["trainingset"], 'training/')

print 'Chosen parameters :\n'

for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

print '\n-----------------------------------------------------------------\n'
print 'Training model ' + str(dir_name) + '...\n'

################################################### Training phase ###################################################

# Note : possible values for gpu: None, '/gpu:0'. If used withing CUDA_VISIBLE_DEVICES=?, then it will use gpu number ? by default
from AxonDeepSeg.train_network import train_model
train_model(path_trainingset, path_model, config_network, path_model_init=path_model_init,gpu='/gpu:0')